# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [126]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [127]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [128]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [129]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [130]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10, verbose=None)

Find what features had non-zero weight.

In [131]:
import numpy

In [132]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [133]:
coefficients = model_all.coefficients

In [134]:
print 'number of nonzeros = %d' % (coefficients['value']).nnz()
print_coefficients(model_all)

number of nonzeros = 6
Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [135]:
print coefficients[coefficients['value'] != 0.0].print_rows(num_rows = 18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[6 rows x 4 columns]

None


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [136]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [137]:
def get_best_model(l1_penalty_values, training_set, validation_set, target, features, max_nonzeros = None):
    l1_penalty_rss = []
    min_rss = None
    best_l1_penalty = None
    best_model = None
    for l1_penalty in l1_penalty_values:
        model = graphlab.linear_regression.create(training_set, target=target, features=features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose=None)
        # predict on validation set 
        predicted = model.predict(validation_set)
        # compute squared error
        rss = ((predicted-validation_set[target])**2).sum()
        l1_penalty_rss.append(rss)
        if min_rss is None or rss < min_rss:
            if (max_nonzeros is not None):
                if (model.coefficients['value'].nnz() == max_nonzeros):
                    min_rss = rss
                    best_l1_penalty = l1_penalty
                    best_model = model
            else:
                min_rss = rss
                best_l1_penalty = l1_penalty
                best_model = model
    print min_rss, best_l1_penalty
    return min_rss, best_l1_penalty, best_model

In [138]:
l1_penalty_values = numpy.logspace(1, 7, num=13)
(min_rss, best_l1_penalty, best_model) = get_best_model(l1_penalty_values, training, validation, 'price', all_features)

6.25766285142e+14 10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [139]:
answers = {}
answers['1'] = best_l1_penalty
answers['2'] = min_rss
print answers

{'1': 10.0, '2': 625766285142459.9}


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [140]:
print 'number of nonzeros = %d' % (best_model.coefficients['value']).nnz()
print_coefficients(best_model)

number of nonzeros = 18
Learned polynomial for degree 17:
       17         16         15         14        13        12
56.07 x  + 9.434 x  + 122.4 x  + 43.29 x  + 6207 x  + 6609 x 
              11             10             9            8         7
 + 9.331e+04 x  + 6.019e+05 x  + 1.292e+04 x + 2.12e+04 x + 148.3 x
             6        5         4             3       2
 + 0.003484 x + 1125 x + 39.12 x + 2.541e+04 x + 937 x + 7937 x + 1.899e+04


In [141]:
answers['3'] = best_model.coefficients['value'].nnz()
print answers

{'1': 10.0, '3': 18, '2': 625766285142459.9}


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [142]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [143]:
l1_penalty_values = numpy.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [144]:
l1_penalty_nnz = []
largest_nnz = None
largest_penalty = None
smallest_nnz = None
smallest_penalty = None
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose=None)
    nnz = (model.coefficients['value']).nnz()
    l1_penalty_nnz.append(nnz)
    if nnz > max_nonzeros and (largest_penalty is None or l1_penalty > largest_penalty):
        largest_nnz = nnz
        largest_penalty = l1_penalty
    if nnz < max_nonzeros and (smallest_penalty is None or l1_penalty < smallest_penalty):
        smallest_nnz = nnz
        smallest_penalty = l1_penalty
print largest_nnz, largest_penalty, smallest_nnz, smallest_penalty

10 2976351441.63 6 3792690190.73


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [145]:
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


In [146]:
l1_penalty_min = largest_penalty
l1_penalty_max = smallest_penalty

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [147]:
answers['4'] = l1_penalty_min
answers['5'] = l1_penalty_max
print answers

{'1': 10.0, '3': 18, '2': 625766285142459.9, '5': 3792690190.7322536, '4': 2976351441.6313133}


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [148]:
l1_penalty_values = numpy.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [149]:
(min_rss, best_l1_penalty, best_model) = get_best_model(l1_penalty_values, training, validation, 'price', all_features, max_nonzeros)

1.04693748875e+15 3448968612.16


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [154]:
coefficients = best_model.coefficients
non_zero_coefficients = coefficients[coefficients['value'] != 0.0]['name']
print coefficients[coefficients['value'] != 0.0].print_rows(num_rows = 18)
print non_zero_coefficients

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

None
['(intercept)', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ]


In [152]:
answers['6'] = best_l1_penalty
answers['7'] = non_zero_coefficients
print answers

{'1': 10.0, '3': 18, '2': 625766285142459.9, '5': 3792690190.7322536, '4': 2976351441.6313133, '7': dtype: str
Rows: ?
['(intercept)', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ], '6': 3448968612.1634369}
